In [45]:
import datetime
import zoneinfo
import email
from email.parser import BytesParser
from email import policy
filepath = 'Your Potential Timetable.eml'
import re
import uuid
from tkinter import *
from tkinter import ttk

In [21]:
with open(filepath) as email_file:
    email_message = email.message_from_file(email_file)
content = (email_message.get_payload())

In [22]:
content1 = content.replace('=\n','')
new_p = re.compile('<td.*?>.*?</td>')

In [23]:
lst1 = new_p.findall(content1)
# print(lst1)
lst1 = lst1[:-1]
# print(len(lst1))

['<td>Enrolled</td>', '<td>33076</td>', '<td>WR03</td>', '<td>COMP SCI</td>', '<td>7211</td>', "<td style=3D'border: 1px solid red; background-color: #febebe; font-weight: 500;'>0</td>", '<td>M 5:00pm - 7:00pm - Ingkarni Wardli 234<br />W 5:00pm - 7:00pm - Ingkarni Wardli 234<br />F 5:00pm - 7:00pm - Ingkarni Wardli 234</td>', '<td>26/05/2025 - 18/08/2025<br />28/05/2025 - 20/08/2025<br />30/05/2025 - 22/08/2025</td>', '<td>North Terrace Campus</td>', '<td>Enrolled</td>', '<td>33089</td>', '<td>LE01</td>', '<td>COMP SCI</td>', '<td>7211</td>', '<td>9</td>', '<td>M 10:00am - 11:00am - Engineering Nth N158<br />W 10:00am - 11:00am - Engineering Nth N158</td>', '<td>26/05/2025 - 18/08/2025<br />28/05/2025 - 20/08/2025</td>', '<td>North Terrace Campus</td>', '<td>Enrolled</td>', '<td>33290</td>', '<td>WR02</td>', '<td>COMP SCI</td>', '<td>7314</td>', '<td>10</td>', '<td>Th 1:00pm - 3:00pm - Engineering & Mathematics EM105<br />Th 1:00pm - 3:00pm - Engineering & Mathematics EM105<br />Th 1:

In [24]:
clear_p = re.compile('</*?td.*?>')
# clear_p2 = re.compile('</td>')
for i in range(len(lst1)):
    lst1[i] = re.sub(clear_p,'',lst1[i])
# print(lst1)

In [25]:
# reshape the list to a 2d array
lst2 = []
for i in range(9):
    lst2.append([])
    for j in range(len(lst1)//9):
        lst2[i].append(lst1[9*j+i])

In [26]:
# sort all the codes in pandas dataframe
# df.columns = ['status','class','section','subject','course','seats open','duration&locations','dates','campus']
time_loc = lst2[6]
for i in range(len(time_loc)):
    time_loc[i] = time_loc[i].split('<br />')
# print(time_loc)
# time_loc[i][j]

[['M 5:00pm - 7:00pm - Ingkarni Wardli 234', 'W 5:00pm - 7:00pm - Ingkarni Wardli 234', 'F 5:00pm - 7:00pm - Ingkarni Wardli 234'], ['M 10:00am - 11:00am - Engineering Nth N158', 'W 10:00am - 11:00am - Engineering Nth N158'], ['Th 1:00pm - 3:00pm - Engineering & Mathematics EM105', 'Th 1:00pm - 3:00pm - Engineering & Mathematics EM105', 'Th 1:00pm - 3:00pm - Engineering & Mathematics EM105', 'Th 1:00pm - 3:00pm - Engineering & Mathematics EM105', 'Th 1:00pm - 3:00pm - Engineering & Mathematics EM105', 'Th 1:00pm - 3:00pm - Engineering & Mathematics EM105'], ['T 11:00am - 1:00pm - Lower Napier LG29'], ['T 5:00pm - 7:00pm - Ingkarni Wardli G22'], ['M 4:00pm - 5:00pm - Napier 208'], ['Th 3:00pm - 4:00pm - Ingkarni Wardli B23'], ['W 3:00pm - 4:30pm - Engineering & Mathematics EM108']]


In [27]:
dura = lst2[7]
for i in range(len(dura)):
    dura[i] = dura[i].split('<br />')


In [28]:
# print(dura)

[['26/05/2025 - 18/08/2025', '28/05/2025 - 20/08/2025', '30/05/2025 - 22/08/2025'], ['26/05/2025 - 18/08/2025', '28/05/2025 - 20/08/2025'], ['14/08/2025 - 14/08/2025', '31/07/2025 - 31/07/2025', '17/07/2025 - 17/07/2025', '3/07/2025 - 3/07/2025', '19/06/2025 - 19/06/2025', '5/06/2025 - 5/06/2025'], ['27/05/2025 - 19/08/2025'], ['19/08/2025 - 19/08/2025'], ['26/05/2025 - 18/08/2025'], ['29/05/2025 - 21/08/2025'], ['23/07/2025 - 23/07/2025']]


In [29]:
# start time & end time -> datetime.datetime
def duration(duradate:str):
    duradate = duradate.split(' - ')
    start = datetime.datetime.strptime(duradate[0], '%d/%m/%Y')
    end = datetime.datetime.strptime(duradate[1], '%d/%m/%Y')
    return(start,end)

dura_date = []
for i in range(len(dura)):
    for j in range(len(dura[i])):
        dura[i][j] = duration(dura[i][j])

# print(dura)
# dura[i][j][0] = starttime,dura[i][j][1] = endtime
# i = course,j = time duration

[[(datetime.datetime(2025, 5, 26, 0, 0), datetime.datetime(2025, 8, 18, 0, 0)), (datetime.datetime(2025, 5, 28, 0, 0), datetime.datetime(2025, 8, 20, 0, 0)), (datetime.datetime(2025, 5, 30, 0, 0), datetime.datetime(2025, 8, 22, 0, 0))], [(datetime.datetime(2025, 5, 26, 0, 0), datetime.datetime(2025, 8, 18, 0, 0)), (datetime.datetime(2025, 5, 28, 0, 0), datetime.datetime(2025, 8, 20, 0, 0))], [(datetime.datetime(2025, 8, 14, 0, 0), datetime.datetime(2025, 8, 14, 0, 0)), (datetime.datetime(2025, 7, 31, 0, 0), datetime.datetime(2025, 7, 31, 0, 0)), (datetime.datetime(2025, 7, 17, 0, 0), datetime.datetime(2025, 7, 17, 0, 0)), (datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 3, 0, 0)), (datetime.datetime(2025, 6, 19, 0, 0), datetime.datetime(2025, 6, 19, 0, 0)), (datetime.datetime(2025, 6, 5, 0, 0), datetime.datetime(2025, 6, 5, 0, 0))], [(datetime.datetime(2025, 5, 27, 0, 0), datetime.datetime(2025, 8, 19, 0, 0))], [(datetime.datetime(2025, 8, 19, 0, 0), datetime.datetime(2

In [30]:
# print(time_loc[0][0])

M 5:00pm - 7:00pm - Ingkarni Wardli 234


In [31]:
# seperate time & location
p2 = r"^([MTWFS][a-z]*)\s([\d:amp\s]+)-([\d:amp\s]+)-\s(.+)$"
match = re.match(p2,time_loc[0][0])
# print(match.group(3))

 7:00pm 


In [32]:

weeks = []
time_start = []
time_end = []
location = []
for i in range(len(time_loc)):
    weeks.append([])
    time_start.append([])
    time_end.append([])
    location.append([])
    for j in range(len(time_loc[i])):
        match = re.match(p2,time_loc[i][j])
        weeks[i].append(match.group(1))
        time_start[i].append(match.group(2).strip())
        time_end[i].append(match.group(3).strip())
        location[i].append(match.group(4))


In [37]:
# start = datetime.datetime(2021,2,28,17,0,0)
# start.strftime('%Y%m%dT%H%M%S')

'20210228T170000'

In [46]:
# create event for one course
def create_event(course:str,start_date,end_date,start_time,end_time,location,campus):
    day = start_date
    course_events = ''
    while day <= end_date:
        event = "\nBEGIN:VEVENT"
        event += f"\nSUMMARY:{course}"
        start = datetime.datetime.strptime((str(day.date())+' '+start_time),'%Y-%m-%d %I:%M%p')
        event += f"\nDTSTART:{start.strftime('%Y%m%dT%H%M%S')}"
        end = datetime.datetime.strptime((str(day.date())+' '+end_time),'%Y-%m-%d %I:%M%p')
        event += f"\nDTEND:{end.strftime('%Y%m%dT%H%M%S')}"
        event += f"\nUID:{uuid.uuid4()}"
        event += f"\nCREATED:{datetime.datetime.now().strftime('%Y%m%dT%H%M%S')}"
        event += f"\nDESCRIPTION:{campus}"
        event += f"\nLOCATION:{location}"
        day += datetime.timedelta(days = 7)
        event += "\nEND:VEVENT"
        course_events += event
    return course_events

In [47]:
with open('Your_timetable.ics','w') as f:
    cal = 'BEGIN:VCALENDAR'
    for i in range(len(lst2[3])):
        for j in range(len(dura[i])):
            cal += create_event(lst2[3][i]+' '+ lst2[4][i],dura[i][j][0],dura[i][j][1],time_start[i][j],time_end[i][j],location[i][j],lst2[8][i])
    cal += '\nEND:VCALENDAR'
    f.write(cal)

In [35]:
#cal = create_event(df['subject'][0]+df['course'][0],dura[0][0][0],dura[0][0][1],time_start[0][0],time_end[0][0],location[0][0],df['campus'][0])
#with open('Your_timetable.ics','wb') as f:
#    f.write(cal.to_ical())